In [1]:
from protosc.simulation import create_categorical_data, compare_models, create_simulation_data, create_correlated_data
from protosc.wrapper import Wrapper
from protosc.filter_model import filter_model, train_xvalidate, select_features
from protosc.feature_matrix import FeatureMatrix
from protosc.parallel import execute_parallel
from protosc.shell_all_models import execute
import pandas as pd
import numpy as np
import random

In [2]:
X, y, ground_truth = create_categorical_data(min_dev=10, max_dev=20, n_categories=3, seed=1234)
print(f'ground truth #features: {len(ground_truth["selected_features"])}')
# X, y, ground_truth = create_correlated_data()

ground truth #features: 25


### Run all models

In [3]:
out = execute(X, y)
pd.DataFrame(out)

100%|██████████| 8/8 [00:18<00:00,  2.26s/it]


,filter,fast_wrapper,slow_wrapper,random,pseudo
features,"[[176, 288, 91, 42, 379, 80, 167, 487, 403, 16...","[[176, 91, 167, 426, 99, 31], [451, 274, 74, 1...","[[91, 171], [196], [74, 423, 274, 226, 451, 33...","[[179, 460, 259, 491, 69, 203, 310, 312, 60, 2...","[[179, 460, 259, 491, 203, 310, 312, 60, 442, ..."
accuracy,"[0.29411764705882354, 0.2807017543859649, 0.37...","[0.47058823529411764, 0.5087719298245614, 0.53...","[0.5490196078431373, 0.45614035087719296, 0.55...","[0.45098039215686275, 0.3684210526315789, 0.33...","[0.47058823529411764, 0.3684210526315789, 0.37..."
recurring,"[91, 288, 423, 451]",[],[],[],[]


## Compare models
* Percentage of features correct: n_correct_selected/(n_correct_selected+n_false_selected)
* Percentage of features found: n_correct_selected/n_total_features
* Percentage of bias found: selected_bias/total_bias

In [4]:
compare = compare_models(out, ground_truth, mean=True)
pd.DataFrame(compare)

,filter,fast_wrapper,slow_wrapper,random,pseudo
%corr_feat,0.318705,0.543885,0.118056,0.060147,0.015802
%feat_found,0.515000,0.465000,0.085000,0.100000,0.025000
%bias_found,0.569306,0.520694,0.094444,0.108194,0.022500
mean_acc,0.319107,0.540785,0.560448,0.352669,0.353475


### Filter model

In [5]:
# Run filter method
filter_res = filter_model(X, y, fold_seed = 1234)
df = pd.DataFrame(filter_res)
df = df.rename(columns={0: 'Features', 1: 'Accuracy'})
df

,Features,Accuracy
0,"[176, 288, 91, 42, 379, 80, 167, 487, 403, 16,...",0.294118
1,"[451, 274, 74, 122, 185, 231, 281, 423, 83, 22...",0.280702
2,"[74, 423, 274, 226, 451, 334, 185, 281, 83, 12...",0.370370
3,"[42, 91, 288, 218, 229, 407, 274, 151, 304, 27...",0.395833
4,"[423, 274, 451, 74, 185, 226, 281, 83, 122, 33...",0.333333
5,"[185, 274, 451, 423, 334, 240, 281, 74, 226, 3...",0.274510
6,"[74, 451, 185, 274, 423, 226, 334, 231, 122, 2...",0.307692
7,"[423, 274, 226, 194, 451, 185, 83, 74, 334, 28...",0.296296


### Wrapper model
#### Fast method (add immediately when cluster increases accuracy)

In [6]:
# Run fast wrapper method (add immediately = True)
fast = Wrapper(X, y, n=10, add_im=True, fold_seed = 1234)
out_fast = fast.wrapper(n_jobs=-1)

# Print outcome in dataframe
df = pd.DataFrame([value for key, value in out_fast.items() if key != 'recurring']).T
df = df.rename(columns={0: 'Model', 1: 'Features', 2: 'Accuracy', 3: 'Accuracy'})
try:
    df['Recurring features'] = [out_fast['recurring']] * len(df)
except KeyError:
    pass
df

100%|██████████| 8/8 [00:03<00:00,  2.24it/s]


,Model,Features,Accuracy
0,"[[176], [91], [167], [426], [99], [31]]","[176, 91, 167, 426, 99, 31]",0.470588
1,"[[451, 274, 74, 122, 185, 231, 281, 423, 83, 2...","[451, 274, 74, 122, 185, 231, 281, 423, 83, 22...",0.491228
2,"[[74, 423, 274, 226, 451, 334, 185, 281, 83, 1...","[74, 423, 274, 226, 451, 334, 185, 281, 83, 12...",0.537037
3,"[[42], [91], [288], [218], [425], [74]]","[42, 91, 288, 218, 425, 74]",0.520833
4,"[[423, 274, 451, 74, 185, 226, 281, 83, 122, 3...","[423, 274, 451, 74, 185, 226, 281, 83, 122, 33...",0.473684
5,"[[185, 274, 451, 423, 334, 240, 281, 74, 226, ...","[185, 274, 451, 423, 334, 240, 281, 74, 226, 3...",0.431373
6,"[[74, 451, 185, 274, 423, 226, 334, 231, 122, ...","[74, 451, 185, 274, 423, 226, 334, 231, 122, 2...",0.487179
7,"[[423, 274, 226, 194, 451, 185, 83, 74, 334, 2...","[423, 274, 226, 194, 451, 185, 83, 74, 334, 28...",0.611111


#### Slow method (add cluster with highest accuracy)

In [7]:
# Run fast wrapper method (add immediately = False)
slow = Wrapper(X, y, n=10, fold_seed = 1234)
out_slow = slow.wrapper(n_jobs=-1)

# Print outcome in dataframe
df = pd.DataFrame([value for key, value in out_slow.items() if key != 'recurring']).T
df = df.rename(columns={0: 'Model', 1: 'Features', 2: 'Accuracy', 3: 'Accuracy'})
try:
    df['Recurring features'] = [out_slow['recurring']] * len(df)
except KeyError:
    pass
df

100%|██████████| 8/8 [00:03<00:00,  2.10it/s]


,Model,Features,Accuracy
0,"[[91], [171]]","[91, 171]",0.549020
1,[[196]],[196],0.456140
2,"[[74, 423, 274, 226, 451, 334, 185, 281, 83, 1...","[74, 423, 274, 226, 451, 334, 185, 281, 83, 12...",0.444444
3,[[453]],[453],0.520833
4,"[[99], [42], [304], [28]]","[99, 42, 304, 28]",0.614035
5,"[[366], [182], [497]]","[366, 182, 497]",0.568627
6,"[[249], [321]]","[249, 321]",0.589744
7,"[[176], [288], [99], [56]]","[176, 288, 99, 56]",0.629630
